In [1]:
import pandas as pd
import numpy as np
from gurobipy import *
from sklearn.cluster import KMeans

In [2]:
BUILD_COST = 40
CAPACITY = 1000

In [3]:
k = 5
D = pd.read_csv("./data/2.6/Demand.txt", delimiter="\t", header=None).to_numpy()
kmean = KMeans(k)
kmean.fit(D[:, 1:3])

centers = kmean.cluster_centers_

with open("./data/2.6/Facility.txt", "w") as temp:
    for id, i in enumerate(centers):
        print(id + 1, *np.round(i, 3), CAPACITY, BUILD_COST, file=temp)

with open("./data/2.6/TransCost.txt", "w") as temp:
    for j in centers:
        for i in D:
            print(np.round(np.linalg.norm(j - i[1:3]), 3), end=" ", file=temp)
        print("", file=temp)

In [4]:
f = pd.read_csv("./data/2.6/Facility.txt", delimiter=" ", header=None).dropna(axis=1).iloc[:, 1:].to_numpy()
t = pd.read_csv("./data/2.6/TransCost.txt", delimiter=" ", header=None).dropna(axis=1).to_numpy()
D = pd.read_csv("./data/2.6/Demand.txt", delimiter="\t", header=None).to_numpy()

In [5]:
model = Model("2.6.3")
x = model.addMVar(t.shape, vtype=GRB.CONTINUOUS, name="x")
c = model.addMVar((f.shape[0]), vtype=GRB.BINARY, name="c")
model.update()

model.addConstr(x.sum(axis=1) <= CAPACITY * c)
model.addConstr(x.sum(axis=0) >= D[:, -1])

model.setObjective((c * BUILD_COST).sum() + (x * t).sum(), sense=GRB.MINIMIZE)

model.optimize()

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2395694
Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: AMD Ryzen 7 5700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2395694 - for non-commercial use only - registered to tr___@gmail.com
Optimize a model with 205 rows, 1005 columns and 2005 nonzeros
Model fingerprint: 0x20df8cf7
Variable types: 1000 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e-01, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+00, 4e+01]
Presolve time: 0.01s
Presolved: 205 rows, 1005 columns, 2005 nonzeros
Variable types: 1000 continuous, 5 integer (5 binary)
Found heuristic solution: objective 4190.6010000

Root rela

In [6]:
if model.status == GRB.Status.OPTIMAL:
    with open("./out/2.6.out", "w") as temp:
        print(0, end=" ", file=temp)
        for i in range(f.shape[0]):
            print(int(abs(np.round(c[i].X))), end=" ", file=temp)
        print("", file=temp)
        for i in range(f.shape[0]):
            if c[i].X:
                for j in range(D.shape[0]):
                    if x[i, j].X != 0:
                        print(f"{i + 1} {j + 1} {x[i, j].X}", file=temp)
        print(model.ObjVal, file=temp)